<a href="https://colab.research.google.com/github/SWhite4han/LLM_based_Customer_Intent_Understanding/blob/main/LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Setup packages
! pip install git+https://github.com/openai/whisper.git
! pip install summa
! pip install opencc-python-reimplemented

In [2]:
# 套件管理
import difflib
import json
import jieba
import numpy as np
import os
import pandas as pd
import requests
import time
import whisper

from google.colab import data_table
from google.colab import files as colab_files
from google.colab import userdata
from opencc import OpenCC
from summa import keywords
from summa.summarizer import summarize

# LLM 摘要

#### 單次呼叫即可達成，因此選用API CALL
function:
*   Making a API call from Google AI with prompt



source:
*   https://colab.research.google.com/github/google/generative-ai-docs/blob/main/site/en/tutorials/python_quickstart.ipynb#scrollTo=ab9ASynfcIZn
*   https://blog.jiatool.com/posts/gemini_api/
*   https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/use-cases/education/use_cases_for_education.ipynb






In [11]:
def call_google_apt(target_text):
  #print(target_text)
  url = f"https://generativelanguage.googleapis.com/v1/models/gemini-pro:generateContent?key={userdata.get('API_KEY')}"
  headers = {'Content-Type': 'application/json'}
  data = {
      "contents": [
          {
              "parts": [{"text": target_text}]
          }
      ]
  }
  response = requests.post(url, headers=headers, json=data)
  print(f"response status_code: {response.status_code}")
  return response.json() #json.dumps(response.json(), indent=4, ensure_ascii=False)

def get_summary_backup(raw_text):
  return call_google_apt(f'請閱讀下列電話客服與客戶之間的對話內容，從[信用卡,信用貸款,房貸,基金,網路銀行]中選出客戶的詢問意圖比較符合的金融商品主題，並做出對話內容以內的繁體中文摘要。回答格式"主題："\n"摘要："：{raw_text}')

def get_summary(raw_text):
  return call_google_apt(f'請閱讀下列電話客服與客戶之間的對話內容，做出對話內容以內的繁體中文摘要。回答格式"摘要："：{raw_text}')

def get_product(raw_text):
  return call_google_apt(f'請閱讀下列電話客服與客戶之間的對話內容，從[信用卡,信用貸款,房貸,基金,網路銀行]中選出客戶的詢問意圖比較符合的金融商品主題。回答格式"主題："：{raw_text}')

def parse_result(api_rlt):
  return api_rlt['candidates'][0]['content']['parts'][0]['text']

# 非LLM摘要

*   (X) CKIP
*   (V) jieba
*   (V) summa textrank:https://github.com/summanlp/textrank
*   (--) tensorflow-text-summarization:https://github.com/dongjun-Lee/text-summarization-tensorflow
*   (--) huggingface:https://huggingface.co/learn/nlp-course/zh-TW/chapter7/5?fw=tf


In [12]:
def replace_punctuation(text):
  text = text.replace('，', ',')
  text = text.replace('。', '.\n')
  text = text.replace('！', '!\n')
  text = text.replace('？', '?\n')
  text = text.replace('；', ';\n')
  text = text.replace('：', ':')
  return text

def replace_space(text):
  text = text.replace('，', ' ')
  text = text.replace('。', ' ')
  text = text.replace('！', ' ')
  text = text.replace('？', ' ')
  text = text.replace('；', ' ')
  text = text.replace('：', ' ')
  text = text.replace(',', ' ')
  text = text.replace('.', ' ')
  text = text.replace('!', ' ')
  text = text.replace('?', ' ')
  text = text.replace(';', ' ')
  text = text.replace(':', ' ')
  return text

def trad_summarize(stt_text):
  words = jieba.cut(replace_punctuation(stt_text), cut_all=False)
  words = [str(word) for word in words if not str(word) in chinese_stop_words]

  # print(keywords.keywords(' '.join(words), split=True))
  return '\n'.join(summarize(replace_punctuation(''.join(words)), ratio=0.3, split=True))

def match_product(doc):
  prod_list = ['信用貸款', '信用卡', '房貸', '基金', '網路銀行']
  prod_idx = []
  for prod in prod_list:
    matcher = difflib.SequenceMatcher(None, prod, doc)
    similarity = matcher.ratio()
    prod_idx.append(similarity)
    # print(similarity)
  return prod_list[np.argmax(prod_idx)]

# 上傳中文及英文測試語音檔案

#### 這邊使用手動上傳方式上傳語音檔案
語音檔案下載位置
*   https://drive.google.com/drive/folders/1jPWk3mf_NvkubCNWWNZYust0bmrs5CiP?usp=sharing

model
*   https://github.com/openai/whisper#available-models-and-languages

source
*   https://huggingface.co/openai/whisper-tiny
*   https://ttsmaker.com/zh-hk

In [5]:
uploaded = colab_files.upload()
files = []
for f in uploaded.keys():
  print('{name} with length {length} bytes'.format(name=f, length=len(uploaded[f])))
  files.append(f)

Saving ttsmaker-file-2024-2-1-20-10-4.mp3 to ttsmaker-file-2024-2-1-20-10-4.mp3
Saving ttsmaker-file-2024-2-1-21-40-0.mp3 to ttsmaker-file-2024-2-1-21-40-0.mp3
Saving ttsmaker-file-2024-2-1-21-42-44.mp3 to ttsmaker-file-2024-2-1-21-42-44.mp3
Saving ttsmaker-file-2024-2-1-21-57-39.mp3 to ttsmaker-file-2024-2-1-21-57-39.mp3
Saving ttsmaker-file-2024-2-4-1-47-25.mp3 to ttsmaker-file-2024-2-4-1-47-25.mp3
Saving ttsmaker-file-2024-2-4-1-49-4.mp3 to ttsmaker-file-2024-2-4-1-49-4.mp3
Saving ttsmaker-file-2024-2-4-20-43-17.mp3 to ttsmaker-file-2024-2-4-20-43-17.mp3
ttsmaker-file-2024-2-1-20-10-4.mp3 with length 466669 bytes
ttsmaker-file-2024-2-1-21-40-0.mp3 with length 377136 bytes
ttsmaker-file-2024-2-1-21-42-44.mp3 with length 609840 bytes
ttsmaker-file-2024-2-1-21-57-39.mp3 with length 439200 bytes
ttsmaker-file-2024-2-4-1-47-25.mp3 with length 418464 bytes
ttsmaker-file-2024-2-4-1-49-4.mp3 with length 340655 bytes
ttsmaker-file-2024-2-4-20-43-17.mp3 with length 488973 bytes


# 模型及其他參數設定

In [6]:
# tiny, base, small, medium, large, large-v2, large-v3
model = whisper.load_model('small')
cc = OpenCC('s2t')
data_table.enable_dataframe_formatter()

chinese_stop_words = [
    '的', '了', '和', '是', '就', '都', '而', '及', '與',
    '著', '或', '一個', '沒有', '我們', '你們', '妳們',
    '他們', '她們', '是否', '我', '您', '謝謝', '再見'
]

100%|████████████████████████████████████████| 461M/461M [00:02<00:00, 170MiB/s]


# 檔案處理

In [7]:
results = []
for f in files:
  name = os.path.basename(f)
  result = model.transcribe(name, prompt="您好，敝姓盧，很高興為您服務")
  results.append(result)
results = [result['text'] for result in results]
result_trans = [cc.convert(result) for result in results]
# # show
# result_trans

# 傳統文本處理

In [14]:
trad_summarys = list(map(trad_summarize, result_trans))
trad_product = list(map(match_product, trad_summarys))
# show
print(trad_product)
trad_summarys

['信用卡', '網路銀行', '信用卡', '基金', '房貸', '信用貸款', '信用貸款']


['爲維護雙方權益,本次對話我們將全程錄音.\n您好,必幸魯,很高興爲服務.\n您好,我想詢問一下關於信用卡羅杜,可以進行調整.\n先生怎摩稱呼呢?林.\n林先生您好,請問方便跟合對一下基本資料嗎?可以.\n請問林先生身份證字後4號是多少呢?4322號的,.',
 '你好,畢竟禮,很高興爲服務.\n你好,不好意思,網絡銀行密碼忘記,請問能幫處理嗎?小姐怎麼稱呼呢?雙木林.\n好,可以連接到,.',
 '你好,陳先生,很抱歉,聽到信用卡遺失.\n請問陳先生身份證字後4號是多少呢?6977,好,.\n好,.\n,陳先生.\n同時,會爲辦理信用卡遺失包案程序,併發送一張新信用卡給.\n同時,請密切監控賬單,以確保沒有爲授權交易.\n不客氣,陳先生.',
 '您好,我想詢問一下定期定額基金扣款成功.\n您好,先生怎麼稱呼呢?口天無.\n請問吳先生身份政治後4號是多少呢?7749,好,.\n好,.',
 '您好,有興趣辦理房屋貸款,想問一下建價方法.\n您好,感謝詢問.\n辦理房屋貸款通常需要進行房屋建價.\n提供不同建價方式,包括委託第三方建價公司進行評估使用自家建價流程.',
 '你好,想借錢,有什麼方式可以借?請問有什麼狀況需要集用嗎?我家提前有些鋼筋外露問題,請師傅評估大概需要20萬元.\n您好,感謝詢問.\n有特定借款金額用途嗎?那想要信用貸款.',
 "Hello, I'm interested in applying for a credit card.\nYou can visit our official website and find the credit card application section.\nOnce there, you'll need to fill out the required information.\nIs there anything specific you'd like assistance with during the application?\nNo, I think I can manage the online application.\nI'll proceed with the online application."]

# LLM 文本處理

In [13]:
llm_prod = []
llm_summarys = []
for stt_text in results:
  prod = get_product(stt_text)
  rlt = get_summary(stt_text)
  llm_prod.append(match_product(parse_result(prod).split('：')[1]))
  llm_summarys.append(parse_result(rlt).split('：')[1]) #['candidates'][0]['content']['parts'][0]['text']
  time.sleep(1)
# show
print(llm_prod)
llm_summarys

response status_code: 200
response status_code: 200
response status_code: 200
response status_code: 200
response status_code: 200
response status_code: 200
response status_code: 200
response status_code: 200
response status_code: 200
response status_code: 200
response status_code: 200
response status_code: 200
response status_code: 200
response status_code: 200
['信用卡', '網路銀行', '信用卡', '基金', '房貸', '信用貸款', '信用卡']


['林先生致電客服詢問信用卡額度調整事項。客服核對基本資料後，告知林先生目前無法立即調整額度，但將會提交請求並轉交相關部門，通常會在一個至兩個工作日內與林先生聯繫。客服確認林先生的聯繫電話後，結束通話。',
 '林小姐致電銀行客服，表示忘記了網路銀行的密碼，希望客服協助重設。客服核對了林小姐的基本資料，包括身分證字號、信用卡最近繳款地點，以及就讀的國小。核對無誤後，客服將重設密碼的通知寄送至林小姐的簡訊中。林小姐確認密碼已重設成功，便向客服道謝並結束通話。',
 '陳先生發現他的信用卡遺失，於是致電尋求客服協助。客服確認陳先生的身份資訊後，將該信用卡設為未暫停以防止進一步的損失，並為陳先生辦理信用卡遺失掛失程序，並寄送一張新的卡片給陳先生。客服建議陳先生通知所有可能使用該卡的自動支付或授權付款的商家，並密切監視帳單以確保沒有未經授權的交易。客服也提到會發送一封電子郵件提供信用卡的詳細資訊，包括到期日和新的安全碼。陳先生完成與客服的通話，並表示若有其他問題或進一步的協助需求，會與客服聯繫。',
 '吳先生致電必幸菜詢問定期定額基金扣款是否成功。客服人員核對吳先生的基本資料，包括身份證字號後四碼7749、信用卡繳款地點是超商，確認基金扣款已成功完成。吳先生表示沒有其他問題需要詢問。',
 '客戶有興趣辦理房屋貸款，並詢問建價方法。客服人員解釋可以用第三方建價公司或建商自己的方式建價，並協助安排建價流程。客戶選擇使用第三方建價公司，提供房屋地址和聯繫方式，客服將安排建價流程，完成後會聯絡客戶並提供相關房貸資訊。',
 '顧客詢問借款方式，客服提供不同的借款方式，包括信用貸款、信用卡分期等，顧客選擇了信用貸款。客服提供線上申請和行動應用程式辦理方式，顧客選擇線上申請。客服告知顧客需親自提供資料，無法直接代為辦理，顧客要求客服儘快提供申請連結。',
 '客戶致電客服詢問申請信用卡的流程。客服人員告知客戶可以透過線上平台或親臨分行辦理，客戶偏好線上申請。客服人員提供線上申請的步驟和所需文件，包括收入證明、身份證明和銀行帳戶證明。申請提交後，審核團隊會進行審核，若核准，會與客戶聯繫後續事宜。']

# 輸出整理
# https://adamsanalysis.com/data-science/ml-system-design


In [15]:
# fake ID
ids = ['A0' + str(np.random.randint(11111111, 99999999)) for i in range(len(files))]
d = {'ID': ids, 'stt_rlt': result_trans, 'trad_products':trad_product, 'trad_summarys':trad_summarys, 'llm_products':llm_prod, 'llm_summarys':llm_summarys}
df = pd.DataFrame(data=d)
# show results
df

,ID,stt_rlt,trad_products,trad_summarys,llm_products,llm_summarys
0,A095964174,"爲了維護雙方的權益,本次對話我們將全程錄音。謝謝。您好,必幸魯,很高興爲您服務。您好,我想詢...",信用卡,"爲維護雙方權益,本次對話我們將全程錄音.\n您好,必幸魯,很高興爲服務.\n您好,我想詢問一...",信用卡,林先生致電客服詢問信用卡額度調整事項。客服核對基本資料後，告知林先生目前無法立即調整額度，但...
1,A095912022,"現在忙線中,請在線上稍後一下。你好,畢竟禮,很高興爲您服務。你好,不好意思,我網絡銀行的密碼...",網路銀行,"你好,畢竟禮,很高興爲服務.\n你好,不好意思,網絡銀行密碼忘記,請問能幫處理嗎?小姐怎麼稱...",網路銀行,林小姐致電銀行客服，表示忘記了網路銀行的密碼，希望客服協助重設。客服核對了林小姐的基本資料，...
2,A023271797,"你好,必興許,很高興爲你服務。你好,我發現我的信用卡不見了,我懷疑它可能遺失了。我應該怎麼處...",信用卡,"你好,陳先生,很抱歉,聽到信用卡遺失.\n請問陳先生身份證字後4號是多少呢?6977,好,....",信用卡,陳先生發現他的信用卡遺失，於是致電尋求客服協助。客服確認陳先生的身份資訊後，將該信用卡設為未...
3,A037463061,"爲了維護雙方的權益,本次對話我們將全程錄音。謝謝。您好,必幸菜,很高興爲您服務。您好,我想詢...",基金,"您好,我想詢問一下定期定額基金扣款成功.\n您好,先生怎麼稱呼呢?口天無.\n請問吳先生身份...",基金,吳先生致電必幸菜詢問定期定額基金扣款是否成功。客服人員核對吳先生的基本資料，包括身份證字號後...
4,A082036838,"您好,我有興趣辦理房屋貸款,想問一下建價的方法。您好,感謝您的詢問。辦理房屋貸款通常需要進行...",房貸,"您好,有興趣辦理房屋貸款,想問一下建價方法.\n您好,感謝詢問.\n辦理房屋貸款通常需要進行...",房貸,客戶有興趣辦理房屋貸款，並詢問建價方法。客服人員解釋可以用第三方建價公司或建商自己的方式建價...
5,A029132088,"你好,我想借錢,有什麼方式可以借?請問有什麼狀況需要集用嗎?我家提前有些鋼筋外露的問題,請師...",信用貸款,"你好,想借錢,有什麼方式可以借?請問有什麼狀況需要集用嗎?我家提前有些鋼筋外露問題,請師傅評...",信用貸款,顧客詢問借款方式，客服提供不同的借款方式，包括信用貸款、信用卡分期等，顧客選擇了信用貸款。客...
6,A013535960,"Hello, I'm interested in applying for a credi...",信用貸款,"Hello, I'm interested in applying for a credit...",信用卡,客戶致電客服詢問申請信用卡的流程。客服人員告知客戶可以透過線上平台或親臨分行辦理，客戶偏好線...
